In [4]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.5/327.5 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.8 MB/s eta 0:00:00


In [1]:
!rm -f minsearch.py
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-06-27 12:34:13--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8002::154, 2606:50c0:8001::154, 2606:50c0:8000::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8002::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3,7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3,74K  --.-KB/s    in 0s      

2024-06-27 12:34:13 (44,3 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [2]:
import requests 
import minsearch

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [3]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

Install ollama and serve phi3 model
```sh
curl -fsSL https://ollama.com/install.sh | sh

ollama start
ollama serve phi3
```

In [6]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [7]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    response = client.chat.completions.create(
        model='phi3',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [8]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [9]:
rag("Write that this is a test.")

" I apologize for any inconvenience, but please note that this is a test environment and not real course material. Thank you!\n\n(Note: Since it's mentioned as a test environment, there isn't an actual answer to the question based on the given context from FAQ database.)"

In [10]:
print(_)

 I apologize for any inconvenience, but please note that this is a test environment and not real course material. Thank you!

(Note: Since it's mentioned as a test environment, there isn't an actual answer to the question based on the given context from FAQ database.)


To run it in docker
```sh
docker run -it \
    -v ollama:/root/.ollama \
    -p 11434:11434 \
    --name ollama \
    ollama/ollama
```

Pull phi3 model
```sh
docker exec -it ollama bash
ollama pull phi3
```

In [12]:
rag("Write that this is a test.")

" I'm sorry, but as an AI, I can't create actual tests or execute code. However, for educational purposes if we were to simulate a test scenario based on provided instructions and FAQs, it might look something like:\n\n```python\n# This is a simulated test written according to the given context of using Git and GitHub in coursework.\ndef test_git_setup():\n    assert 'Git' in CONTEXT  # Simulate checking for existence of git reference in context\n\ndef test_github_repo_cloning():\n    assert 'clone the course repo' in CONTEXT  # Simulating a check if cloning process is mentioned in FAQs\n```"

In [13]:
print(_)

 I'm sorry, but as an AI, I can't create actual tests or execute code. However, for educational purposes if we were to simulate a test scenario based on provided instructions and FAQs, it might look something like:

```python
# This is a simulated test written according to the given context of using Git and GitHub in coursework.
def test_git_setup():
    assert 'Git' in CONTEXT  # Simulate checking for existence of git reference in context

def test_github_repo_cloning():
    assert 'clone the course repo' in CONTEXT  # Simulating a check if cloning process is mentioned in FAQs
```
